## Qiskit Aqua Gradient Framework

In [ ]:
import numpy as np
from qiskit.aqua.operators import X, I 
from qiskit.aqua.operators.gradients import Gradient, NaturalGradient, QFI, Hessian

### First Order Gradients

In [ ]:
H = 0.5 * X - 1 * Z
a = Parameter('a')
b = Parameter('b')
params = [a, b]

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rz(params[0], q[0])
qc.rx(params[1], q[0])
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)



#### Gradients w.r.t. Measurement Operator Coefficients

#### Gradients w.r.t. Measurement Quantum State Parameters

##### Parameter Shift Parameters

In [ ]:
state_grad = Gradient().convert(operator=op, params=params, method='param_shift')

##### Finite Difference Parameters

In [14]:
state_grad = Gradient().convert(operator=op, params=params, method='finite_diff')

NameError: name 'Gradient' is not defined

##### Linear Combination of Unitaries Parameters

In [ ]:
state_grad = Gradient().convert(operator=op, params=params, method='lin_comb')

#### Natural Gradient

### Second Order Gradients

#### Gradients w.r.t. Measurement Operator Coefficients

#### Gradients w.r.t. Measurement Quantum State Parameters

##### Parameter Shift Parameters

##### Linear Combination of Unitaries Parameters

### QFI

#### Full QFI
Using linear combination of unitaries

#### Block-diagonal Approximation
Without working qubits
Requires unrolling into Pauli rotations and unparameterized Gates

#### Diagonal Approximation
Without working qubits
Requires unrolling into Pauli rotations and unparameterized Gates

### Application Examples

#### VQE with first and second order gradient based optimization

#### Variational Quantum Imaginary Time Evolution (VarQITE)


#### QBMs? - see if time allows